In [31]:
import pylab as plt
import numpy as np
import pandas as pd
import scipy
from scipy import interpolate
from scipy.interpolate import InterpolatedUnivariateSpline

import re
import sys

import matplotlib.pyplot as plt
import random
try:
    import cPickle as pickle
except ImportError:
    import pickle
from scipy.interpolate import interp2d, NearestNDInterpolator,LinearNDInterpolator

In [32]:
ei = 0
spacing=25

NumToRun=1000
LinInterps=[]
NDInterps={}

eps=0.11   # This is a trick to stop divide by zero errors

# energies=[100,300,400,450,500,550,600,700,800,900,1000,2000,4000,6000]
energies=[6000]
energies=[100, 300, 400, 450, 475, 500, 525, 550, 575, 600, 
          625, 650, 675, 700, 725, 750, 775, 800, 825, 850, 875, 
          900, 950,  1000, 1100, 1200, 1300, 1400, 1500, 1600,
          1700, 1800, 1900, 2000, 3000, 4000, 5000, 6000]

def GetMuonInfo(startMuon):
    depthMuon   = np.round(depth(*startMuon),2)
    distMuon    = np.round(rocklength(*startMuon),2)
    thMuon      = np.round(np.arctan(   (startMuon[0]**2+startMuon[1]**2 +eps)**0.5/(depthMuon+eps)   ),2)

    # if (depthMuon == 0):
    #     print("Muon depth is Zero", thMuon, startMuon[0], startMuon[1], depthMuon )


    phiMuon     = np.round(np.arctan2((startMuon[1]+eps),(startMuon[0]+eps)),2)
    return thMuon,phiMuon,depthMuon,distMuon

In [33]:
# To open rocklength from edge of mountain to detector
f=open("Files/rocklength_to_detector.pkl",'rb')
rocklength=pickle.load(f)
f.close() 

#for depth of mountain above the detector
f=open("Files/ExtendedMountain.pkl",'rb')
depth=pickle.load(f)    
f.close()

/var/folders/tr/r0bv1gl52_vb7v8bswkb_z0nb301gc/T/ipykernel_67881/1260163091.py:3: DeprecationWarning: Please use `RectBivariateSpline` from the `scipy.interpolate` namespace, the `scipy.interpolate.fitpack2` namespace is deprecated.
  rocklength=pickle.load(f)
/var/folders/tr/r0bv1gl52_vb7v8bswkb_z0nb301gc/T/ipykernel_67881/1260163091.py:8: DeprecationWarning: Please use `Delaunay` from the `scipy.spatial` namespace, the `scipy.spatial.qhull` namespace is deprecated.
  depth=pickle.load(f)


In [34]:
intexs=[]
inteys=[]
intepercs=[]

for X in range(-525,1500,spacing):
    for Y in range(-2500,750,spacing):
        intexs.append(X)
        inteys.append(Y)

In [35]:
for energy in energies:

    print("Energy: ", energy)

    # Load in spline for survival fraction for each energy at a given x, y position. 
    # f= open('../RegeneratedFiles/Files/Proposal_Muons_interpolator'+str(energy)+'GeV.pkl', 'rb')
    f= open('Files/outputs/Proposal_Muons_interpolator'+str(energy)+'GeV.pkl', 'rb')
    PercentMuons = pickle.load(f)
    f.close()
    
    ei+=1
    
    costhetas=[]
    phis=[]
    survfrac=[]


    intepercs=[] 

    print("Interpolating Muon Survival Fractions...")
    for X in range(-525,1500,spacing):        
        for Y in range(-2500,750,spacing):  
            # Get the interpolated survival fraction at a given x, y postion
            intepercs.append(PercentMuons(X,Y)) 

    intepercs = np.array(intepercs)      
    intepercs=intepercs.reshape(len(np.unique(intexs)), len(np.unique(inteys)))


    # Get Muon info for each event
    print("Getting Muon Info...")
    for i in range(0,len(intexs)):
        x=np.array(intexs).flatten()[i]
        y=np.array(inteys).flatten()[i]
        c=np.array(intepercs).flatten()[i]
        
        startMuon = (x,y)
        
        thMuon,phiMuon,depthMuon,distMuon=GetMuonInfo(startMuon)
        
        # if(np.cos(thMuon)>0.15):
        survfrac.append(c)
        costhetas.append(np.cos(thMuon))
        phis.append(phiMuon)

        survfrac = [0 if x != x else x for x in survfrac]
        costhetas = [0 if x != x else x for x in costhetas]
        phis = [0 if x != x else x for x in phis]

    #Add shifted phis to satisfy periodic boundary condition
    costhetas=np.concatenate([costhetas,costhetas,costhetas,2-np.array(costhetas)])
    phis=np.concatenate([phis,np.array(phis)-2*np.pi,np.array(phis)+2*np.pi,phis])
    survfrac=np.concatenate([survfrac, survfrac, survfrac,survfrac])
    maxcol=max(np.array(survfrac).flatten())

    print(costhetas)

    print("Interpolating...")
    intereff_lin=LinearNDInterpolator(np.array([costhetas,phis]).transpose(),np.array(survfrac).flatten(),rescale=True)

    # LinInterps[energy]=intereff_lin
    LinInterps.append(intereff_lin)
    
f=open("../Files/LinInterps.pkl",'wb')
pickle.dump(LinInterps,f)
f.close()


/var/folders/tr/r0bv1gl52_vb7v8bswkb_z0nb301gc/T/ipykernel_67881/3658877375.py:8: DeprecationWarning: Please use `RectBivariateSpline` from the `scipy.interpolate` namespace, the `scipy.interpolate.fitpack2` namespace is deprecated.
  PercentMuons = pickle.load(f)


Energy:  100
Interpolating Muon Survival Fractions...
Getting Muon Info...
[0.         0.         0.         ... 1.87949723 1.87949723 1.87949723]
Interpolating...
Energy:  300
Interpolating Muon Survival Fractions...
Getting Muon Info...
[0.         0.         0.         ... 1.87949723 1.87949723 1.87949723]
Interpolating...
Energy:  400
Interpolating Muon Survival Fractions...
Getting Muon Info...
[0.         0.         0.         ... 1.87949723 1.87949723 1.87949723]
Interpolating...
Energy:  450
Interpolating Muon Survival Fractions...
Getting Muon Info...
[0.         0.         0.         ... 1.87949723 1.87949723 1.87949723]
Interpolating...
Energy:  475
Interpolating Muon Survival Fractions...
Getting Muon Info...
[0.         0.         0.         ... 1.87949723 1.87949723 1.87949723]
Interpolating...
Energy:  500
Interpolating Muon Survival Fractions...
Getting Muon Info...
[0.         0.         0.         ... 1.87949723 1.87949723 1.87949723]
Interpolating...
Energy:  525
Int